### python .xml to .json

In [ ]:
!pip install xmltodict

In [ ]:
import json
import xmltodict

with open("/content/drive/MyDrive/Lotte_sample/10054_60_s_9_meta.xml",'r') as f:
    xmlString = f.read()

jsonString = json.dumps(xmltodict.parse(xmlString), indent=4)
 
print("\nJSON output(output.json):")
print(jsonString)


JSON output(output.json):
{
    "comp_cd": {
        "identifier": {
            "identifier": "10054_60_s_9.jpg",
            "copyright": "LOTTE Data Communication Company"
        },
        "div_cd": {
            "item_cd": "03021200001",
            "item_no": "10054",
            "div_l": "\uc8fc\ub958",
            "div_m": "\uae30\ud0c0\uc8fc\ub958",
            "div_s": "\uce75\ud14c\uc77c",
            "div_n": "\uce75\ud14c\uc77c",
            "comp_nm": "\ub86f\ub370\uc8fc\ub958",
            "img_prod_nm": "\uc21c\ud558\ub9ac\uc18c\ub2e4\ud1a1\ubc14\ub098\ub098355ML",
            "volume": "355ML",
            "barcd": "8801030996231",
            "width": "7",
            "length": "7",
            "height": "12",
            "nutrition_info": "{\"1\ud68c\uc81c\uacf5\ub7c9\":\"\",\"\ucd1d\ub0b4\uc6a9\ub7c9(g)\":\"\",\"\ucd1d\ub0b4\uc6a9\ub7c9(mL)\":\"\",\"\uc5d0\ub108\uc9c0(\u3389)\":\"\",\"\ub2e8\ubc31\uc9c8(g)\":\"\",\"\uc9c0\ubc29(g)\":\"\",\"\ud0c4\uc218\ud654\ubb3c

### Image Copy & Paste Example

In [ ]:
import numpy as np
import cv2

## paste image
to_paste = np.full((2988, 2988, 3), 255, dtype=np.uint8) # 계산대 색으로 조정
print(to_paste.shape)

## copy image
to_copy = cv2.imread('/content/drive/MyDrive/Lotte_sample/10094_0_m_1.jpg', cv2.IMREAD_COLOR) 
print(to_copy.shape)
# reshape
imageHeight, imageWidth = to_copy.shape[:2]

resizeHeight = int(0.25 * imageHeight) # x 0.25 resize !!
resizeWidth = int(0.25 * imageWidth)

to_copy_resize = cv2.resize(to_copy, (resizeHeight, resizeWidth), interpolation = cv2.INTER_CUBIC)

## copy & paste
to_paste[500:500+resizeHeight, 500:500+resizeWidth] = to_copy_resize 

cv2.imwrite("/content/drive/MyDrive/Lotte_sample_result/final_img.jpg", to_paste)

(2988, 2988, 3)
(2988, 2988, 3)


True

### Sample Dataset Unzip

In [ ]:
%cd /content/drive/MyDrive/LOTTE_sample

/content/drive/MyDrive/LOTTE_sample


In [ ]:
!unzip -qq "/content/drive/MyDrive/LOTTE_sample/LOTTE_sample.zip"

### Data Augmentation Code (using Jacard Overlap)

In [ ]:
!pip install xmltodict # xml2json 에 필요한 라이브러리 설치

## code for Jacard Overlap (IOU)
def find_intersection(set_1, set_2):
    """
    Find the intersection of every box combination between two sets of boxes that are in boundary coordinates.

    :param set_1: set 1, a tensor of dimensions (n1, 4)
    :param set_2: set 2, a tensor of dimensions (n2, 4)
    :return: intersection of each of the boxes in set 1 with respect to each of the boxes in set 2, a tensor of dimensions (n1, n2)
    """

    # PyTorch auto-broadcasts singleton dimensions
    lower_bounds = torch.max(set_1[:, :2].unsqueeze(1), set_2[:, :2].unsqueeze(0))  # (n1, n2, 2)
    upper_bounds = torch.min(set_1[:, 2:].unsqueeze(1), set_2[:, 2:].unsqueeze(0))  # (n1, n2, 2)
    intersection_dims = torch.clamp(upper_bounds - lower_bounds, min=0)  # (n1, n2, 2)
    return intersection_dims[:, :, 0] * intersection_dims[:, :, 1]  # (n1, n2)

def find_jaccard_overlap(set_1, set_2):
    """
    Find the Jaccard Overlap (IoU) of every box combination between two sets of boxes that are in boundary coordinates.

    :param set_1: set 1, a tensor of dimensions (n1, 4)
    :param set_2: set 2, a tensor of dimensions (n2, 4)
    :return: Jaccard Overlap of each of the boxes in set 1 with respect to each of the boxes in set 2, a tensor of dimensions (n1, n2)
    """

    # Find intersections
    intersection = find_intersection(set_1, set_2)  # (n1, n2)

    # Find areas of each box in both sets
    areas_set_1 = (set_1[:, 2] - set_1[:, 0]) * (set_1[:, 3] - set_1[:, 1])  # (n1)
    areas_set_2 = (set_2[:, 2] - set_2[:, 0]) * (set_2[:, 3] - set_2[:, 1])  # (n2)

    # Find the union
    # PyTorch auto-broadcasts singleton dimensions
    union = areas_set_1.unsqueeze(1) + areas_set_2.unsqueeze(0) - intersection  # (n1, n2)

    return intersection / union  # (n1, n2)

In [ ]:
import numpy as np
import cv2
import json
import xmltodict
import glob
from random import *
import math
import torch

# to make _meta.xml information list
file_path = "/content/drive/MyDrive/Lotte_sample/*" 
file_pathes = glob.glob(file_path)
file_list = [file for file in file_pathes if file.endswith("_meta.xml")]
# print ("file_list_py: {}".format(file_list_py))


## 예시) 9개의 images & annotations -> 1개의 final image & annotation

## to_paste image
to_paste = np.full((2988, 2988, 3), 255, dtype=np.uint8) # 계산대 색으로 조정
print(to_paste.shape)

# number of objects
n_objects = 5
# resize ratio
resize_ratio = 0.4

# # starting offset -> 3x3
# offset = []
# for i in range (int(math.sqrt(n_objects))):
#     for j in range (int(math.sqrt(n_objects))):
#         height = i * int(2988/math.sqrt(n_objects))  
#         width = j * int(2988/math.sqrt(n_objects))
#         offset.append([height, width])

pre_boxes = torch.zeros(1, 4, dtype = torch.long) # IOU 비교를 위한 boxes 
for i in range (n_objects):

    # ver_idx, hor_idx = offset[i]

    # # random offset
    # ver_idx = randint(ver_idx, ver_idx + int(2988/(math.sqrt(n_objects)*2)))
    # hor_idx = randint(hor_idx, hor_idx + int(2988/(math.sqrt(n_objects)*2)))
    # print(ver_idx, hor_idx)

    line = file_list[i]

    # .xml to .json 
    with open(line,'r') as f:
        xmlString = f.read()
    obj_Info = json.loads(json.dumps(xmltodict.parse(xmlString), indent=4))

    # to_copy image
    to_copy = cv2.imread(file_path[:-1] + obj_Info['comp_cd']['annotation']['filename'], cv2.IMREAD_COLOR) 

    imageHeight, imageWidth = to_copy.shape[:2]

    resizeHeight = int(resize_ratio * imageHeight)
    resizeWidth = int(resize_ratio * imageWidth)

    to_copy_resize = cv2.resize(to_copy, (resizeHeight, resizeWidth), interpolation = cv2.INTER_CUBIC)
   
    # to_copy_resize image 속 object 별로 진행
    boxes_dict = []
    if 'dict' in str(type(obj_Info['comp_cd']['annotation']['object'])): # 이미지 속 물체가 1개일 경우, 주석 다름
        boxes_dict.append(obj_Info['comp_cd']['annotation']['object']) # 인위적으로 리스트 안으로 넣어줌
    else: boxes_dict =  obj_Info['comp_cd']['annotation']['object'] 

    boxes = []
    for k, obj in enumerate (boxes_dict):

        dict_val_box = obj['bndbox'].values()
        list_box = list(dict_val_box)
        org_box = list(map(int, list_box))
        box = [round(j*resize_ratio) for j in org_box] 
        boxes.append(box)
    
    # random offset + IOU->occlusion 확인
    while True:
        ## 랜덤 좌표
        ver_idx = randint(0, 2000)
        hor_idx = randint(0, 2000) 

        ver = ver_idx
        hor = hor_idx

        new_boxes = []
        for xmin, ymin, xmax, ymax in boxes:
            x_min = hor
            y_min = ver
            x_max = hor + (xmax - xmin)
            y_max = ver + (ymax - ymin)
            new_boxes.append([x_min, y_min, x_max, y_max])
            
            hor = hor + (xmax - xmin)

        # overlap 검사 실시
        new_boxes = torch.tensor(new_boxes)
        overlap = find_jaccard_overlap(pre_boxes, new_boxes).view([-1])
        overlap = overlap.tolist()

        check = 1
        for a in overlap: 
            if a != 0: check = 0 
        if check == 1: break # occlusion 발생하지 않으면 -> while 문 탈출
    
    # overlap 비교를 위한 pre_boxes -> new_boxes 추가
    pre_boxes = torch.cat([pre_boxes, new_boxes], dim=0)
    
    # import pdb;pdb.set_trace()

    # new_boxes = new_boxes.tolist() # Tensor -> list
    # image copy & paste
    for box in boxes:
        to_paste[ver_idx:ver_idx+(box[3]-box[1]), hor_idx:hor_idx+(box[2]-box[0])] = \
                                                    to_copy_resize[box[1]:box[3], box[0]:box[2]]
        # 같은 이미지 속의 물체들은 서로 붙여놓음
        hor_idx = hor_idx+(box[2]-box[0])


    # category = obj_Info['comp_cd']['div_cd']['item_cd'] # label
    # image_path = file_path[:-1] + obj_Info['comp_cd']['annotation']['filename'] # path
    # resolution = obj_Info['comp_cd']['annotation']['size']
    # break

# save final image
cv2.imwrite("/content/drive/MyDrive/Lotte_sample_result/final_img.jpg", to_paste)

(2988, 2988, 3)


True

### Test Code

In [ ]:
import numpy as np
import cv2
import json
import xmltodict
import glob
from random import *
import math
import torch

# to make _meta.xml information list
file_path = "/content/drive/MyDrive/Lotte_sample/*" 
file_pathes = glob.glob(file_path)
file_list = [file for file in file_pathes if file.endswith("_meta.xml")]
# print ("file_list_py: {}".format(file_list_py))


## 예시) 9개의 images & annotations -> 1개의 final image & annotation

## to_paste image
to_paste = np.full((2988, 2988, 3), 255, dtype=np.uint8) # 계산대 색으로 조정
print(to_paste.shape)

# number of objects
n_objects = 5
# resize ratio
resize_ratio = 0.4

# # starting offset -> 3x3
# offset = []
# for i in range (int(math.sqrt(n_objects))):
#     for j in range (int(math.sqrt(n_objects))):
#         height = i * int(2988/math.sqrt(n_objects))  
#         width = j * int(2988/math.sqrt(n_objects))
#         offset.append([height, width])

pre_boxes = torch.zeros(1, 4, dtype = torch.long) # IOU 비교를 위한 boxes 
for i in range (n_objects):

    # ver_idx, hor_idx = offset[i]

    # # random offset
    # ver_idx = randint(ver_idx, ver_idx + int(2988/(math.sqrt(n_objects)*2)))
    # hor_idx = randint(hor_idx, hor_idx + int(2988/(math.sqrt(n_objects)*2)))
    # print(ver_idx, hor_idx)

    line = file_list[i]

    # .xml to .json 
    with open(line,'r') as f:
        xmlString = f.read()
    obj_Info = json.loads(json.dumps(xmltodict.parse(xmlString), indent=4))

    # to_copy image
    to_copy = cv2.imread(file_path[:-1] + obj_Info['comp_cd']['annotation']['filename'], cv2.IMREAD_COLOR) 

    imageHeight, imageWidth = to_copy.shape[:2]

    resizeHeight = int(resize_ratio * imageHeight)
    resizeWidth = int(resize_ratio * imageWidth)

    to_copy_resize = cv2.resize(to_copy, (resizeHeight, resizeWidth), interpolation = cv2.INTER_CUBIC)
   
    # to_copy_resize image 속 object 별로 진행
    boxes_dict = []
    if 'dict' in str(type(obj_Info['comp_cd']['annotation']['object'])): # 이미지 속 물체가 1개일 경우, 주석 다름
        boxes_dict.append(obj_Info['comp_cd']['annotation']['object']) # 인위적으로 리스트 안으로 넣어줌
    else: boxes_dict =  obj_Info['comp_cd']['annotation']['object'] 

    boxes = []
    for k, obj in enumerate (boxes_dict):

        dict_val_box = obj['bndbox'].values()
        list_box = list(dict_val_box)
        org_box = list(map(int, list_box))
        box = [round(j*resize_ratio) for j in org_box] 
        boxes.append(box)
    
    # random offset + IOU->occlusion 확인
    while True:
        ## 랜덤 좌표
        ver_idx = randint(0, 2000)
        hor_idx = randint(0, 2000) 

        ver = ver_idx
        hor = hor_idx

        new_boxes = []
        for xmin, ymin, xmax, ymax in boxes:
            x_min = hor
            y_min = ver
            x_max = hor + (xmax - xmin)
            y_max = ver + (ymax - ymin)
            new_boxes.append([x_min, y_min, x_max, y_max])
            
            hor = hor + (xmax - xmin)

        # overlap 검사 실시
        new_boxes = torch.tensor(new_boxes)
        overlap = find_jaccard_overlap(pre_boxes, new_boxes).view([-1])
        overlap = overlap.tolist()

        check = 1
        for a in overlap: 
            if a != 0: check = 0 
        if check == 1: break # occlusion 발생하지 않으면 -> while 문 탈출
    
    # overlap 비교를 위한 pre_boxes -> new_boxes 추가
    pre_boxes = torch.cat([pre_boxes, new_boxes], dim=0)
    
    # import pdb;pdb.set_trace()

    # new_boxes = new_boxes.tolist() # Tensor -> list
    # image copy & paste
    for box in boxes:
        to_paste[ver_idx:ver_idx+(box[3]-box[1]), hor_idx:hor_idx+(box[2]-box[0])] = \
                                                    to_copy_resize[box[1]:box[3], box[0]:box[2]]
        # 같은 이미지 속의 물체들은 서로 붙여놓음
        hor_idx = hor_idx+(box[2]-box[0])


    # category = obj_Info['comp_cd']['div_cd']['item_cd'] # label
    # image_path = file_path[:-1] + obj_Info['comp_cd']['annotation']['filename'] # path
    # resolution = obj_Info['comp_cd']['annotation']['size']
    # break

# save final image
cv2.imwrite("/content/drive/MyDrive/Lotte_sample_result/final_img.jpg", to_paste)

(2988, 2988, 3)


True

False